In [ ]:
!pip uninstall torch torchvision torchaudio
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
import torch
print(torch.__version__)
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
print(f"CUDA 디바이스 수: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    print(f"현재 CUDA 디바이스: {torch.cuda.current_device()}")
    print(f"CUDA 디바이스 이름: {torch.cuda.get_device_name(0)}")

In [4]:
# GPU 강제 활성화된 메모리 최적화 HRNet 뇌종양 세그멘테이션 모델 (5GB 미만 메모리 사용)
# 필요한 라이브러리 임포트
import os
import gc
import cv2
import numpy as np
import matplotlib.pyplot as plt
import warnings
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor
from albumentations import Compose, Resize, Normalize, HorizontalFlip, VerticalFlip, RandomRotate90
from albumentations.pytorch import ToTensorV2
from sklearn.metrics import f1_score, precision_score, recall_score
from datasets import load_dataset
import timm

# GPU 메모리 제한 설정 - 5GB 미만 사용
if torch.cuda.is_available():
    # 메모리 제한 설정 (바이트 단위, 5GB = 5 * 1024 * 1024 * 1024)
    torch.cuda.set_per_process_memory_fraction(0.6)  # GPU 메모리의 60%만 사용
    print(f"GPU 메모리 제한: 전체 메모리의 60%")

warnings.filterwarnings('ignore')

# 메모리 사용량 추적 함수
def print_gpu_memory():
    if torch.cuda.is_available():
        print(f"GPU 메모리 사용량: {torch.cuda.memory_allocated() / (1024**3):.3f} GB / {torch.cuda.memory_reserved() / (1024**3):.3f} GB")
    else:
        print("GPU를 사용할 수 없습니다. CPU 모드로 실행합니다.")

# 메모리 최적화 함수
def clean_memory():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
# 랜덤 시드 설정
pl.seed_everything(42)

# 데이터셋 로드
print("데이터셋 로드 중...")
dataset = load_dataset("dwb2023/brain-tumor-image-dataset-semantic-segmentation")
print("데이터셋 로드 완료!")

# 데이터 전처리 및 증강
class BrainTumorDataset(Dataset):
    def __init__(self, hf_dataset, split='train', img_size=160):  # 이미지 크기를 160으로 설정
        self.dataset = hf_dataset[split]
        self.img_size = img_size
        self.split = split
        self.transform = self.get_transforms(split)
        
        # 훈련 데이터 크기 제한 - 주석 해제하여 사용 가능
        # if split == 'train':
        #     self.dataset = self.dataset.select(range(min(len(self.dataset), 500)))
        # elif split == 'valid' or split == 'test':
        #     self.dataset = self.dataset.select(range(min(len(self.dataset), 100)))
            
    def get_transforms(self, split):
        if split == 'train':
            # 훈련 세트에는 가벼운 데이터 증강만 적용
            return Compose([
                Resize(self.img_size, self.img_size),
                HorizontalFlip(p=0.5),
                Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
                ToTensorV2(),
            ])
        else:
            # 검증/테스트 세트에는 기본 전처리만 적용
            return Compose([
                Resize(self.img_size, self.img_size),
                Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
                ToTensorV2(),
            ])

    def create_mask_from_segmentation(self, segmentation, height, width):
        # segmentation (다각형 좌표)에서 마스크 생성
        mask = np.zeros((height, width), dtype=np.float32)

        if segmentation and isinstance(segmentation, list):
            for polygon in segmentation:
                if len(polygon) > 4:  # 최소 3개의 좌표(x,y) 쌍이 필요
                    # [x1, y1, x2, y2, ...] 형식을 (N, 2) 형식으로 변환
                    points = np.array(polygon).reshape(-1, 2).astype(np.int32)
                    cv2.fillPoly(mask, [points], 1)

        return mask

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        image = np.array(item['image'])
        height, width = image.shape[:2]

        # segmentation에서 마스크 생성
        mask = self.create_mask_from_segmentation(item['segmentation'], height, width)

        # 이미지 및 마스크 전처리
        transformed = self.transform(image=image, mask=mask)
        image = transformed['image']
        mask = transformed['mask'].unsqueeze(0)  # 채널 차원 추가 [1, H, W]

        return image, mask

# 데이터 로더 생성 (메모리 최적화)
def create_dataloaders(dataset, batch_size=2):  # 배치 크기를 더 줄임
    train_dataset = BrainTumorDataset(dataset, 'train')
    val_dataset = BrainTumorDataset(dataset, 'valid')
    test_dataset = BrainTumorDataset(dataset, 'test')

    # num_workers 및 핀 메모리 설정
    num_workers = 2 if torch.cuda.is_available() else 0
    pin_memory = torch.cuda.is_available()

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                             num_workers=num_workers, pin_memory=pin_memory)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                           num_workers=num_workers, pin_memory=pin_memory)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                            num_workers=num_workers, pin_memory=pin_memory)

    return train_loader, val_loader, test_loader

# Dice 손실 함수 정의
class DiceLoss(nn.Module):
    def __init__(self, smooth=1.0):
        super(DiceLoss, self).__init__()
        self.smooth = smooth

    def forward(self, y_pred, y_true):
        y_pred = torch.sigmoid(y_pred)

        # 분자 (intersection)
        intersection = torch.sum(y_true * y_pred)

        # 분모 (union)
        union = torch.sum(y_true) + torch.sum(y_pred)

        # Dice 계수
        dice = (2.0 * intersection + self.smooth) / (union + self.smooth)

        return 1.0 - dice  # 최소화를 위해 1에서 빼기

# 조합 손실 함수 (간소화)
class CombinedLoss(nn.Module):
    def __init__(self, smooth=1.0):
        super(CombinedLoss, self).__init__()
        self.smooth = smooth

    def forward(self, y_pred, y_true):
        y_pred = torch.sigmoid(y_pred)

        # Dice 손실
        intersection = torch.sum(y_true * y_pred)
        union = torch.sum(y_true) + torch.sum(y_pred)
        dice_loss = 1.0 - (2.0 * intersection + self.smooth) / (union + self.smooth)

        return dice_loss

# 더 가벼운 HRNet 모델 사용 (HRNet-W18 Small 또는 ResNet18)
class LightHRNetSegmentation(pl.LightningModule):
    def __init__(self, num_classes=1, lr=0.001, weight_decay=1e-4):
        super().__init__()
        self.save_hyperparameters()
        self.num_classes = num_classes
        self.lr = lr
        self.weight_decay = weight_decay

        # 간소화된 손실 함수
        self.criterion = CombinedLoss()

        # 성능 메트릭 저장
        self.best_val_f1 = 0.0

        # 사용 가능한 모델 확인
        try:
            # 사용 가능한 HRNet 모델 확인
            available_models = [m for m in timm.list_models() if 'hrnet' in m]
            print(f"사용 가능한 HRNet 모델: {available_models}")
            
            # 가장 가벼운 HRNet 모델 선택
            if 'hrnet_w18_small' in available_models:
                model_name = 'hrnet_w18_small'
            elif 'hrnet_w18' in available_models:
                model_name = 'hrnet_w18'
            else:
                model_name = 'resnet18'  # 대체 모델
                
            print(f"선택된 모델: {model_name}")
            
            if 'hrnet' in model_name:
                self.backbone = timm.create_model(
                    model_name, 
                    pretrained=True,
                    features_only=True
                )
            else:
                # ResNet18 사용
                self.backbone = timm.create_model(
                    'resnet18', 
                    pretrained=True,
                    features_only=True
                )
        except Exception as e:
            print(f"모델 로드 실패: {e}")
            print("ResNet18 모델로 대체합니다.")
            # ResNet18 백업 모델 사용
            self.backbone = timm.create_model(
                'resnet18', 
                pretrained=True,
                features_only=True
            )

        # 마지막 특성 맵의 채널 수
        last_channels = self.backbone.feature_info[-1]['num_chs']

        # 가벼운 세그멘테이션 헤드
        self.segmentation_head = nn.Sequential(
            nn.Conv2d(last_channels, 32, kernel_size=3, padding=1),  # 채널 수 줄임
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, num_classes, kernel_size=1)
        )

    def forward(self, x):
        # 입력 크기 저장
        input_size = x.shape[2:]

        # 백본으로 특성 추출
        features = self.backbone(x)

        # 마지막 특성 맵 사용
        last_feature = features[-1]

        # 세그멘테이션 헤드 통과
        logits = self.segmentation_head(last_feature)

        # 원래 이미지 크기로 업샘플링
        if logits.shape[2:] != input_size:
            logits = F.interpolate(logits, size=input_size, mode='bilinear', align_corners=False)

        return logits

    def training_step(self, batch, batch_idx):
        images, masks = batch
        predictions = self(images)
        loss = self.criterion(predictions, masks)

        # F1 스코어 계산
        pred_masks = (torch.sigmoid(predictions) > 0.5).float()
        f1 = self.calculate_f1_score(pred_masks, masks)

        # 로깅
        self.log('train_loss', loss, prog_bar=True)
        self.log('train_f1', f1, prog_bar=True)

        # 메모리 정리
        if batch_idx % 20 == 0:
            clean_memory()
            
        return loss

    def validation_step(self, batch, batch_idx):
        images, masks = batch
        predictions = self(images)
        loss = self.criterion(predictions, masks)

        # F1 스코어 계산
        pred_masks = (torch.sigmoid(predictions) > 0.5).float()
        f1 = self.calculate_f1_score(pred_masks, masks)
        
        # 로깅
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_f1', f1, prog_bar=True)

        # 최고 F1 스코어 업데이트
        if f1 > self.best_val_f1:
            self.best_val_f1 = f1

        return {'val_loss': loss, 'val_f1': f1}

    def test_step(self, batch, batch_idx):
        images, masks = batch
        predictions = self(images)

        # F1 스코어 계산
        pred_masks = (torch.sigmoid(predictions) > 0.5).float()
        f1 = self.calculate_f1_score(pred_masks, masks)
        precision = self.calculate_precision(pred_masks, masks)
        recall = self.calculate_recall(pred_masks, masks)
        
        # 로깅
        self.log('test_f1', f1)
        self.log('test_precision', precision)
        self.log('test_recall', recall)

        return {'test_f1': f1, 'test_precision': precision, 'test_recall': recall}

    def predict_step(self, batch, batch_idx):
        images, _ = batch
        predictions = self(images)
        pred_masks = (torch.sigmoid(predictions) > 0.5).float()
        return pred_masks

    def calculate_f1_score(self, pred, target):
        # CPU 사용하여 메모리 부담 감소
        pred_flat = pred.view(-1).cpu().detach().numpy()
        target_flat = target.view(-1).cpu().detach().numpy()

        # 예측값이 0.5보다 크면 1, 아니면 0
        pred_binary = (pred_flat > 0.5).astype(np.int32)
        target_binary = (target_flat > 0.5).astype(np.int32)

        # F1 스코어 계산
        f1 = f1_score(target_binary, pred_binary, zero_division=1)
        return torch.tensor(f1)

    def calculate_precision(self, pred, target):
        # CPU 사용하여 메모리 부담 감소
        pred_flat = pred.view(-1).cpu().detach().numpy()
        target_flat = target.view(-1).cpu().detach().numpy()

        pred_binary = (pred_flat > 0.5).astype(np.int32)
        target_binary = (target_flat > 0.5).astype(np.int32)

        precision = precision_score(target_binary, pred_binary, zero_division=1)
        return torch.tensor(precision)

    def calculate_recall(self, pred, target):
        # CPU 사용하여 메모리 부담 감소
        pred_flat = pred.view(-1).cpu().detach().numpy()
        target_flat = target.view(-1).cpu().detach().numpy()

        pred_binary = (pred_flat > 0.5).astype(np.int32)
        target_binary = (target_flat > 0.5).astype(np.int32)

        recall = recall_score(target_binary, pred_binary, zero_division=1)
        return torch.tensor(recall)

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(
            self.parameters(),
            lr=self.lr,
            weight_decay=self.weight_decay
        )

        # 스케줄러
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='max', factor=0.5, patience=3, verbose=True, min_lr=1e-6
        )

        return {
            'optimizer': optimizer,
            'lr_scheduler': scheduler,
            'monitor': 'val_f1',
            'interval': 'epoch'
        }

# 메모리 효율적인 예측 결과 시각화
def visualize_predictions(model, test_loader, threshold=0.5, num_samples=3):
    model.eval()
    device = next(model.parameters()).device

    plt.figure(figsize=(15, 5 * num_samples))

    metrics = {
        'f1_scores': [],
        'precision_scores': [],
        'recall_scores': []
    }

    for i, (images, masks) in enumerate(test_loader):
        if i >= num_samples:
            break

        images = images.to(device)
        masks = masks.to(device)

        with torch.no_grad():
            predictions = model(images)
            predictions = torch.sigmoid(predictions)
            pred_masks = (predictions > threshold).float()

        # 메트릭 계산
        pred_flat = pred_masks[0].cpu().flatten().numpy()
        mask_flat = masks[0].cpu().flatten().numpy()

        f1 = f1_score(mask_flat, pred_flat, zero_division=1)
        precision = precision_score(mask_flat, pred_flat, zero_division=1)
        recall = recall_score(mask_flat, pred_flat, zero_division=1)

        metrics['f1_scores'].append(f1)
        metrics['precision_scores'].append(precision)
        metrics['recall_scores'].append(recall)

        # CPU로 데이터 이동 및 넘파이 변환
        images = images.cpu().numpy()
        masks = masks.cpu().numpy()
        pred_masks = pred_masks.cpu().numpy()

        # 배치의 첫 번째 이미지만 표시
        image = np.transpose(images[0], (1, 2, 0))
        mask = masks[0, 0]
        pred_mask = pred_masks[0, 0]

        # 정규화 이미지 역변환
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        image = std * image + mean
        image = np.clip(image, 0, 1)

        # 시각화
        plt.subplot(num_samples, 3, i*3 + 1)
        plt.imshow(image)
        plt.title(f'이미지 {i+1}')
        plt.axis('off')

        plt.subplot(num_samples, 3, i*3 + 2)
        plt.imshow(mask, cmap='gray')
        plt.title(f'실제 마스크 {i+1}')
        plt.axis('off')

        plt.subplot(num_samples, 3, i*3 + 3)
        plt.imshow(pred_mask, cmap='gray')
        plt.title(f'예측 마스크 {i+1}\nF1: {f1:.4f}')
        plt.axis('off')

        # 중간에 메모리 정리
        clean_memory()

    # 평균 메트릭 계산
    avg_f1 = np.mean(metrics['f1_scores'])
    avg_precision = np.mean(metrics['precision_scores'])
    avg_recall = np.mean(metrics['recall_scores'])

    plt.suptitle(f'평균 F1: {avg_f1:.4f}, 정밀도: {avg_precision:.4f}, 재현율: {avg_recall:.4f}')
    plt.tight_layout()
    plt.savefig('prediction_results_lightweight_optimized.png')
    plt.show()

    return metrics

# 간소화된 TTA(Test Time Augmentation) 함수
def tta_predict(model, image):
    model.eval()

    # 원본 예측
    with torch.no_grad():
        pred_orig = torch.sigmoid(model(image))

    # 수평 뒤집기
    with torch.no_grad():
        img_flip = torch.flip(image, [3])
        pred_flip = torch.sigmoid(model(img_flip))
        pred_flip = torch.flip(pred_flip, [3])

    # 예측 결과 평균 
    final_pred = (pred_orig + pred_flip) / 2.0
    return final_pred

# 메인 함수
def main():
    print("메모리 최적화된 뇌종양 세그멘테이션 - 경량화 모델 (5GB 미만 메모리 사용)")
    print("=" * 50)

    # PyTorch 환경 확인 및 설정
    print(f"PyTorch 버전: {torch.__version__}")
    
    # GPU 상태 확인
    if torch.cuda.is_available():
        print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
        print(f"CUDA 버전: {torch.version.cuda}")
        print(f"CUDA 디바이스 이름: {torch.cuda.get_device_name(0)}")
        print(f"GPU 총 메모리: {torch.cuda.get_device_properties(0).total_memory / (1024**3):.2f} GB")
        
        # GPU 디버깅 정보
        device_count = torch.cuda.device_count()
        print(f"사용 가능한 GPU 개수: {device_count}")
        for i in range(device_count):
            print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
    else:
        print("GPU를 사용할 수 없습니다. CPU 모드로 실행됩니다.")

    # PyTorch 메모리 최적화 설정
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

    # 데이터 로더 생성 (배치 크기 감소)
    batch_size = 2  # 메모리 사용량 감소를 위해 배치 크기 최소화
    train_loader, val_loader, test_loader = create_dataloaders(dataset, batch_size)
    print(f"데이터 로더 생성 완료! 훈련 샘플 수: {len(train_loader.dataset)}")

    # 모델 생성
    print("경량화 모델 생성 중...")
    model = LightHRNetSegmentation(num_classes=1, lr=0.001, weight_decay=1e-4)
    print("모델 생성 완료!")
    
    # 모델 요약정보 출력
    total_params = sum(p.numel() for p in model.parameters())
    print(f"모델 총 파라미터 수: {total_params:,}")
    
    # 체크포인트 저장 최소화
    checkpoint_callback = ModelCheckpoint(
        monitor='val_f1',
        dirpath='checkpoints/',
        filename='lightweight_brain_tumor-{epoch:02d}-{val_f1:.4f}',
        save_top_k=1,  # 최고 성능 모델만 저장
        mode='max',
    )

    early_stop_callback = EarlyStopping(
        monitor='val_f1',
        patience=5,
        mode='max',
        verbose=True,
    )

    # PL 트레이너 설정 (수정됨)
    print("트레이너 설정 중...")
    try:
        # PyTorch Lightning 2.0 이상 버전
        trainer = pl.Trainer(
            max_epochs=15,
            accelerator='auto',  # 자동 감지
            devices='auto' if torch.cuda.is_available() else None,
            precision=16 if torch.cuda.is_available() else 32,
            callbacks=[
                checkpoint_callback,
                early_stop_callback,
                LearningRateMonitor(logging_interval='epoch')
            ],
            log_every_n_steps=20,
            gradient_clip_val=1.0,
            accumulate_grad_batches=4,
            enable_checkpointing=True,
            enable_progress_bar=True,
            enable_model_summary=True,
            deterministic=True,
        )
    except Exception as e:
        print(f"트레이너 초기화 오류: {e}")
        print("대체 설정으로 시도합니다...")
        
        # 대체 설정 (이전 버전 호환)
        try:
            trainer = pl.Trainer(
                max_epochs=15,
                gpus=1 if torch.cuda.is_available() else 0,  # 이전 버전 방식
                precision=16 if torch.cuda.is_available() else 32,
                callbacks=[
                    checkpoint_callback,
                    early_stop_callback,
                    LearningRateMonitor(logging_interval='epoch')
                ],
                log_every_n_steps=20,
                gradient_clip_val=1.0,
                accumulate_grad_batches=4,
                checkpoint_callback=True,
                progress_bar_refresh_rate=10
            )
        except Exception as e2:
            print(f"대체 트레이너 초기화 오류: {e2}")
            print("최소 설정으로 시도합니다...")
            
            # 최소 설정
            trainer = pl.Trainer(
                max_epochs=15,
                logger=False,
                callbacks=[early_stop_callback]
            )

    # 현재 메모리 사용량 확인
    print_gpu_memory()

    # 메모리 정리
    clean_memory()
    
    try:
        # 모델 훈련
        print("모델 훈련 시작...")
        trainer.fit(model, train_loader, val_loader)
        print("훈련 완료!")

        # 최고 성능 출력
        print(f"최고 검증 F1 스코어: {model.best_val_f1:.4f}")

        # 메모리 정리
        clean_memory()
        
        # 모델 테스트
        print("모델 테스트 중...")
        test_results = trainer.test(model, test_loader)
        print(f"테스트 결과: {test_results}")
        
        # 메모리 정리
        clean_memory()

        # 모델 저장
        torch.save(model.state_dict(), 'lightweight_brain_tumor_optimized.pth')
        print("모델 저장 완료: lightweight_brain_tumor_optimized.pth")

        # 메모리 정리
        clean_memory()
        
        # 예측 결과 시각화
        print("예측 결과 시각화 중...")
        metrics = visualize_predictions(model, test_loader, threshold=0.5, num_samples=3)
        print(f"테스트 세트의 평균 F1 스코어: {np.mean(metrics['f1_scores']):.4f}")
        
        print("완료!")
        
    except Exception as e:
        print(f"오류 발생: {e}")
        import traceback
        traceback.print_exc()
        
        if "CUDA out of memory" in str(e):
            print("\nGPU 메모리 부족 오류가 발생했습니다.")
            print("다음 단계를 시도해 보세요:\n")
            print("1. 배치 크기를 1로 줄이기")
            print("2. 이미지 크기를 128 또는 96으로 줄이기")
            print("3. 더 작은 모델(ResNet18) 사용하기")
            print("4. 데이터셋 크기 제한하기 (코드에서 주석 해제)")
            print("5. torch.cuda.set_per_process_memory_fraction() 값을 0.4로 낮추기\n")
        
    finally:
        # 최종 메모리 정리
        clean_memory()
        print("최종 메모리 사용량:")
        print_gpu_memory()

if __name__ == "__main__":
    # 디버그 정보 출력
    print("\n==== GPU 디버그 정보 ====")
    print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"CUDA 디바이스 수: {torch.cuda.device_count()}")
        print(f"현재 디바이스: {torch.cuda.current_device()}")
        print(f"디바이스 이름: {torch.cuda.get_device_name(0)}")
    print("=======================\n")
    
    main()

Seed set to 42


데이터셋 로드 중...
데이터셋 로드 완료!

==== GPU 디버그 정보 ====
CUDA 사용 가능: False

메모리 최적화된 뇌종양 세그멘테이션 - 경량화 모델 (5GB 미만 메모리 사용)
PyTorch 버전: 2.7.0+cpu
GPU를 사용할 수 없습니다. CPU 모드로 실행됩니다.
데이터 로더 생성 완료! 훈련 샘플 수: 1502
경량화 모델 생성 중...
사용 가능한 HRNet 모델: ['hrnet_w18', 'hrnet_w18_small', 'hrnet_w18_small_v2', 'hrnet_w18_ssld', 'hrnet_w30', 'hrnet_w32', 'hrnet_w40', 'hrnet_w44', 'hrnet_w48', 'hrnet_w48_ssld', 'hrnet_w64']
선택된 모델: hrnet_w18_small


Unexpected keys (downsamp_modules.0.0.bias, downsamp_modules.0.0.weight, downsamp_modules.0.1.bias, downsamp_modules.0.1.num_batches_tracked, downsamp_modules.0.1.running_mean, downsamp_modules.0.1.running_var, downsamp_modules.0.1.weight, downsamp_modules.1.0.bias, downsamp_modules.1.0.weight, downsamp_modules.1.1.bias, downsamp_modules.1.1.num_batches_tracked, downsamp_modules.1.1.running_mean, downsamp_modules.1.1.running_var, downsamp_modules.1.1.weight, downsamp_modules.2.0.bias, downsamp_modules.2.0.weight, downsamp_modules.2.1.bias, downsamp_modules.2.1.num_batches_tracked, downsamp_modules.2.1.running_mean, downsamp_modules.2.1.running_var, downsamp_modules.2.1.weight, final_layer.0.bias, final_layer.0.weight, final_layer.1.bias, final_layer.1.num_batches_tracked, final_layer.1.running_mean, final_layer.1.running_var, final_layer.1.weight, classifier.bias, classifier.weight) found while loading pretrained weights. This may be expected if model is being adapted.
Using default `M

모델 생성 완료!
모델 총 파라미터 수: 3,131,681
트레이너 설정 중...
트레이너 초기화 오류: `devices` selected with `CPUAccelerator` should be an int > 0.
대체 설정으로 시도합니다...
대체 트레이너 초기화 오류: Trainer.__init__() got an unexpected keyword argument 'gpus'
최소 설정으로 시도합니다...
GPU를 사용할 수 없습니다. CPU 모드로 실행합니다.
모델 훈련 시작...
오류 발생: ReduceLROnPlateau.__init__() got an unexpected keyword argument 'verbose'
최종 메모리 사용량:
GPU를 사용할 수 없습니다. CPU 모드로 실행합니다.


Traceback (most recent call last):
  File "C:\Users\tjddl\AppData\Local\Temp\ipykernel_9964\1342225758.py", line 588, in main
    trainer.fit(model, train_loader, val_loader)
  File "c:\GitHubRepo\NeuraSeg-HRNet-High-Resolution-MRI-Brain-Tumor-Detection-and-Segmentation\.venv\Lib\site-packages\pytorch_lightning\trainer\trainer.py", line 561, in fit
    call._call_and_handle_interrupt(
  File "c:\GitHubRepo\NeuraSeg-HRNet-High-Resolution-MRI-Brain-Tumor-Detection-and-Segmentation\.venv\Lib\site-packages\pytorch_lightning\trainer\call.py", line 48, in _call_and_handle_interrupt
    return trainer_fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\GitHubRepo\NeuraSeg-HRNet-High-Resolution-MRI-Brain-Tumor-Detection-and-Segmentation\.venv\Lib\site-packages\pytorch_lightning\trainer\trainer.py", line 599, in _fit_impl
    self._run(model, ckpt_path=ckpt_path)
  File "c:\GitHubRepo\NeuraSeg-HRNet-High-Resolution-MRI-Brain-Tumor-Detection-and-Segmentation\.venv\Lib\site-pack